# IBM Watson OpenScale Lab instructions

This notebook should be run in a Watson Studio project, using the **Python 3.6 ** runtime environment. **If you are viewing this in Watson Studio and do not see Python 3.6 in the upper right corner of your screen, please update the runtime now.** It requires service credentials for the following Cloud services:
  * IBM Watson OpenScale
  * Watson Machine Learning
  
If you have a paid Cloud account, you may also provision a **Databases for PostgreSQL** or **Db2 Warehouse** service to take full advantage of integration with Watson Studio and continuous learning services. If you choose not to provision this paid service, you can use the free internal PostgreSQL storage with OpenScale, but will not be able to configure continuous learning for your model.

The notebook will train, create and deploy a German Credit Risk model, configure OpenScale to monitor that deployment, and inject seven days' worth of historical records and measurements for viewing in the OpenScale Insights dashboard.

# Package installation

In [1]:
!rm -rf $PIP_BUILD
!pip install psycopg2-binary | tail -n 1
!pip install --upgrade watson-machine-learning-client --no-cache | tail -n 1
!pip install --upgrade ibm-ai-openscale --no-cache | tail -n 1
!pip install --upgrade numpy --no-cache | tail -n 1
!pip install --upgrade lime --no-cache | tail -n 1
!pip install --upgrade SciPy --no-cache | tail -n 1
!pip install -U pyspark==2.1.2 --no-cache | tail -n 1

Requirement already up-to-date: numpy in /opt/conda/envs/Python36/lib/python3.6/site-packages (1.17.0)


# Provision services and configure credentials

If you have not already, provision an instance of IBM Watson OpenScale using the [OpenScale link in the Cloud catalog](https://cloud.ibm.com/catalog/services/watson-openscale).

Your Cloud API key can be generated by going to the [**Users** section of the Cloud console](https://cloud.ibm.com/iam#/users).  
* From that page, click your name, scroll down to the **API Keys** section, and click **Create an IBM Cloud API key**. 
* Give your key a name and click **Create**, then copy the created key and paste it below.

Alternately, from the [IBM Cloud CLI](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use) :

```bash
ibmcloud login --sso
ibmcloud iam api-key-create 'my_key'
```

In [2]:
CLOUD_API_KEY = "****"

Get the Watson OpenScale GUID using the [IBM Cloud CLI](https://console.bluemix.net/docs/cli/reference/ibmcloud/download_cli.html#install_use) :

```bash
ibmcloud resource service-instance <Watson_OpenScale_instance_name>
```

In [3]:
AIOS_GUID = "****"

In [4]:
AIOS_CREDENTIALS = {
    "instance_guid": AIOS_GUID,
    "apikey": CLOUD_API_KEY,
    "url": "https://api.aiopenscale.cloud.ibm.com"
}

Next you will need credentials for Watson Machine Learning. If you already have a WML instance, you may use credentials for it. To provision a new Lite instance of WML, use the [IBM Cloud catalog](https://cloud.ibm.com/catalog/services/machine-learning):  
* Give your service a name, and click **Create**.  
* Once your instance is created, click the **Service Credentials** link on the left side of the screen.  
* Click the **New credential** button, give your credentials a name, and click **Add**.  
* Your new credentials can be accessed by clicking the **View credentials** button.  
* Copy and paste your WML credentials into the cell below.

In [8]:
WML_CREDENTIALS = {
  "apikey": "****",
  "iam_apikey_description": "Auto-generated for key ****",
  "iam_apikey_name": "wdp-writer",
  "iam_role_crn": "crn:v1:bluemix:public:iam::::serviceRole:Writer",
  "iam_serviceid_crn": "crn:v1:bluemix:public:iam-identity::a/****::serviceid:ServiceId-****",
  "instance_id": "****",
  "url": "https://us-south.ml.cloud.ibm.com"
}

This lab can use Databases for PostgreSQL, Db2 Warehouse, or a free internal verison of PostgreSQL to create a datamart for OpenScale.

    **If you have previously configured OpenScale**, it will use your existing datamart, and not interfere with any models you are currently monitoring. **Do not update the cell below where DB_CREDENTIALS are filled out AND comment it out or do not run it. Instead, uncomment the cell for `DB_CREDENTIALS = NONE` AND RUN IT INSTEAD**.

**If you do not have a paid Cloud account or would prefer not to provision this paid service**, you may use the free internal PostgreSQL service with OpenScale. **Do not update the cell below where DB_CREDENTIALS are filled out AND comment it out or do not run it. Instead, uncomment the cell for `DB_CREDENTIALS = NONE` AND RUN IT INSTEAD**.

**To provision a new instance of Db2 Warehouse**:  
* Locate [Db2 Warehouse in the Cloud catalog](https://cloud.ibm.com/catalog/services/db2-warehouse)  
* Give your service a name, and click **Create**.  
* Once your instance is created, click the **Service Credentials** link on the left side of the screen.  
* Click the **New credential** button, give your credentials a name, and click **Add**.  
* Your new credentials can be accessed by clicking the **View credentials** button.  
* Copy and paste your Db2 Warehouse credentials into the cell below.

**To provision a new instance of Databases for PostgreSQL**:
* Locate [Databases for PostgreSQL in the Cloud catalog](*https://cloud.ibm.com/catalog/services/databases-for-postgresql)  
* Give your service a name, and click **Create**.  
* Once your instance is created, click the **Service Credentials** link on the left side of the screen.  
* Click the **New credential** button, give your credentials a name, and click **Add**.  
* Your new credentials can be accessed by clicking the **View credentials** button.  
* Copy and paste your Databases for PostgreSQL credentials into the cell below.


IF you have previously configured OpenScale and are using an existing datamart OR would prefer not to provision the paid service, uncomment the cell below to set `DB_CREDENTIALS = NONE` AND do not run or comment out the cell where `DB_CREDENTIALS` are set.

In [9]:
DB_CREDENTIALS = None

Comment out the cell below if you have set `DB_CREDENTIALS = None`.

In [ ]:
#DB_CREDENTIALS = {
  "hostname": "***",
  "password": "***",
  "https_url": "***",
  "port": 50000,
  "ssldsn": "***",
  "host": "***",
  "jdbcurl": "***",
  "uri": "***",
  "db": "***",
  "dsn": "****",
  "username": "***",
  "ssljdbcurl": "****"
}

__If you previously configured OpenScale to use the free internal version of PostgreSQL, you can switch to a new datamart using a paid database service.__ If you would like to delete the internal PostgreSQL configuration and create a new one using service credentials supplied in the cell above, set the __KEEP_MY_INTERNAL_POSTGRES__ variable below to __False__ below. In this case, the notebook will remove your existing internal PostgreSQL datamart and create a new one with the supplied credentials. __*NO DATA MIGRATION WILL OCCUR.*__

In [10]:
KEEP_MY_INTERNAL_POSTGRES = True

# Run the notebook

At this point, the notebook is ready to run. You can either run the cells one at a time, or click the **Kernel** option above and select **Restart and Run All** to run all the cells.

# Load and explore data

## Load the training data from github

In [11]:
!rm german_credit_data_biased_training.csv
!wget https://raw.githubusercontent.com/IBM/monitor-wml-model-with-watson-openscale/master/data/german_credit_data_biased_training.csv

--2019-08-07 19:28:33--  https://raw.githubusercontent.com/IBM/monitor-wml-model-with-watson-openscale/master/data/german_credit_data_biased_training.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 689622 (673K) [text/plain]
Saving to: ‘german_credit_data_biased_training.csv’

100%[======================================>] 689,622     --.-K/s   in 0.03s   

2019-08-07 19:28:33 (22.7 MB/s) - ‘german_credit_data_biased_training.csv’ saved [689622/689622]



In [12]:
from pyspark.sql import SparkSession
import pandas as pd
import json

spark = SparkSession.builder.getOrCreate()
pd_data = pd.read_csv("german_credit_data_biased_training.csv", sep=",", header=0)
df_data = spark.read.csv(path="german_credit_data_biased_training.csv", sep=",", header=True, inferSchema=True)
df_data.head()

Row(CheckingStatus='0_to_200', LoanDuration=31, CreditHistory='credits_paid_to_date', LoanPurpose='other', LoanAmount=1889, ExistingSavings='100_to_500', EmploymentDuration='less_1', InstallmentPercent=3, Sex='female', OthersOnLoan='none', CurrentResidenceDuration=3, OwnsProperty='savings_insurance', Age=32, InstallmentPlans='none', Housing='own', ExistingCreditsCount=1, Job='skilled', Dependents=1, Telephone='none', ForeignWorker='yes', Risk='No Risk')

## Explore data

In [13]:
df_data.printSchema()

root
 |-- CheckingStatus: string (nullable = true)
 |-- LoanDuration: integer (nullable = true)
 |-- CreditHistory: string (nullable = true)
 |-- LoanPurpose: string (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- ExistingSavings: string (nullable = true)
 |-- EmploymentDuration: string (nullable = true)
 |-- InstallmentPercent: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- OthersOnLoan: string (nullable = true)
 |-- CurrentResidenceDuration: integer (nullable = true)
 |-- OwnsProperty: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- InstallmentPlans: string (nullable = true)
 |-- Housing: string (nullable = true)
 |-- ExistingCreditsCount: integer (nullable = true)
 |-- Job: string (nullable = true)
 |-- Dependents: integer (nullable = true)
 |-- Telephone: string (nullable = true)
 |-- ForeignWorker: string (nullable = true)
 |-- Risk: string (nullable = true)



In [14]:
print("Number of records: " + str(df_data.count()))

Number of records: 5000


# Create a model

In [15]:
spark_df = df_data
(train_data, test_data) = spark_df.randomSplit([0.8, 0.2], 24)

MODEL_NAME = "Spark German Risk Model - Final"
DEPLOYMENT_NAME = "Spark German Risk Deployment - Final"

print("Number of records for training: " + str(train_data.count()))
print("Number of records for evaluation: " + str(test_data.count()))

spark_df.printSchema()

Number of records for training: 4016
Number of records for evaluation: 984
root
 |-- CheckingStatus: string (nullable = true)
 |-- LoanDuration: integer (nullable = true)
 |-- CreditHistory: string (nullable = true)
 |-- LoanPurpose: string (nullable = true)
 |-- LoanAmount: integer (nullable = true)
 |-- ExistingSavings: string (nullable = true)
 |-- EmploymentDuration: string (nullable = true)
 |-- InstallmentPercent: integer (nullable = true)
 |-- Sex: string (nullable = true)
 |-- OthersOnLoan: string (nullable = true)
 |-- CurrentResidenceDuration: integer (nullable = true)
 |-- OwnsProperty: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- InstallmentPlans: string (nullable = true)
 |-- Housing: string (nullable = true)
 |-- ExistingCreditsCount: integer (nullable = true)
 |-- Job: string (nullable = true)
 |-- Dependents: integer (nullable = true)
 |-- Telephone: string (nullable = true)
 |-- ForeignWorker: string (nullable = true)
 |-- Risk: string (nullable = 

In [16]:
from pyspark.ml.feature import OneHotEncoder, StringIndexer, IndexToString, VectorAssembler
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.ml import Pipeline, Model

si_CheckingStatus = StringIndexer(inputCol = 'CheckingStatus', outputCol = 'CheckingStatus_IX')
si_CreditHistory = StringIndexer(inputCol = 'CreditHistory', outputCol = 'CreditHistory_IX')
si_LoanPurpose = StringIndexer(inputCol = 'LoanPurpose', outputCol = 'LoanPurpose_IX')
si_ExistingSavings = StringIndexer(inputCol = 'ExistingSavings', outputCol = 'ExistingSavings_IX')
si_EmploymentDuration = StringIndexer(inputCol = 'EmploymentDuration', outputCol = 'EmploymentDuration_IX')
si_Sex = StringIndexer(inputCol = 'Sex', outputCol = 'Sex_IX')
si_OthersOnLoan = StringIndexer(inputCol = 'OthersOnLoan', outputCol = 'OthersOnLoan_IX')
si_OwnsProperty = StringIndexer(inputCol = 'OwnsProperty', outputCol = 'OwnsProperty_IX')
si_InstallmentPlans = StringIndexer(inputCol = 'InstallmentPlans', outputCol = 'InstallmentPlans_IX')
si_Housing = StringIndexer(inputCol = 'Housing', outputCol = 'Housing_IX')
si_Job = StringIndexer(inputCol = 'Job', outputCol = 'Job_IX')
si_Telephone = StringIndexer(inputCol = 'Telephone', outputCol = 'Telephone_IX')
si_ForeignWorker = StringIndexer(inputCol = 'ForeignWorker', outputCol = 'ForeignWorker_IX')

In [17]:
si_Label = StringIndexer(inputCol="Risk", outputCol="label").fit(spark_df)
label_converter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=si_Label.labels)

In [18]:
va_features = VectorAssembler(inputCols=["CheckingStatus_IX", "CreditHistory_IX", "LoanPurpose_IX", "ExistingSavings_IX", "EmploymentDuration_IX", "Sex_IX", \
                                         "OthersOnLoan_IX", "OwnsProperty_IX", "InstallmentPlans_IX", "Housing_IX", "Job_IX", "Telephone_IX", "ForeignWorker_IX", \
                                         "LoanDuration", "LoanAmount", "InstallmentPercent", "CurrentResidenceDuration", "LoanDuration", "Age", "ExistingCreditsCount", \
                                         "Dependents"], outputCol="features")

In [19]:
from pyspark.ml.classification import RandomForestClassifier
classifier = RandomForestClassifier(featuresCol="features")

pipeline = Pipeline(stages=[si_CheckingStatus, si_CreditHistory, si_EmploymentDuration, si_ExistingSavings, si_ForeignWorker, si_Housing, si_InstallmentPlans, si_Job, si_LoanPurpose, si_OthersOnLoan,\
                               si_OwnsProperty, si_Sex, si_Telephone, si_Label, va_features, classifier, label_converter])
model = pipeline.fit(train_data)

In [20]:
predictions = model.transform(test_data)
evaluatorDT = BinaryClassificationEvaluator(rawPredictionCol="prediction")
area_under_curve = evaluatorDT.evaluate(predictions)

#default evaluation is areaUnderROC
print("areaUnderROC = %g" % area_under_curve)

areaUnderROC = 0.709704


# Save and deploy the model

In [21]:
from watson_machine_learning_client import WatsonMachineLearningAPIClient
import json

wml_client = WatsonMachineLearningAPIClient(WML_CREDENTIALS)

### Remove existing model and deployment

In [22]:
model_deployment_ids = wml_client.deployments.get_uids()
for deployment_id in model_deployment_ids:
    deployment = wml_client.deployments.get_details(deployment_id)
    model_id = deployment['entity']['deployable_asset']['guid']
    if deployment['entity']['name'] == DEPLOYMENT_NAME:
        print('Deleting deployment id', deployment_id)
        wml_client.deployments.delete(deployment_id)
        print('Deleting model id', model_id)
        wml_client.repository.delete(model_id)
wml_client.repository.list_models()

Deleting deployment id ddc1960a-28a0-4761-a53d-f8ebac95d730
Deleting model id aab903cf-c708-4d19-bd44-0cb23e29bc84
----  ----  -------  ---------
GUID  NAME  CREATED  FRAMEWORK
----  ----  -------  ---------


In [23]:
model_props = {
    wml_client.repository.ModelMetaNames.NAME: "{}".format(MODEL_NAME),
    wml_client.repository.ModelMetaNames.EVALUATION_METHOD: "binary",
    wml_client.repository.ModelMetaNames.EVALUATION_METRICS: [
        {
           "name": "areaUnderROC",
           "value": area_under_curve,
           "threshold": 0.7
        }
    ]
}

In [24]:
wml_models = wml_client.repository.get_details()
model_uid = None
for model_in in wml_models['models']['resources']:
    if MODEL_NAME == model_in['entity']['name']:
        model_uid = model_in['metadata']['guid']
        break

if model_uid is None:
    print("Storing model ...")

    published_model_details = wml_client.repository.store_model(model=model, meta_props=model_props, training_data=train_data, pipeline=pipeline)
    model_uid = wml_client.repository.get_model_uid(published_model_details)
    print("Done")

Storing model ...


/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/opt/conda/envs/Python36/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


Done


In [25]:
model_uid

'fe7a19f2-7550-4a2f-b02e-6cb5c11a3cf0'

In [26]:
wml_deployments = wml_client.deployments.get_details()
deployment_uid = None
for deployment in wml_deployments['resources']:
    if DEPLOYMENT_NAME == deployment['entity']['name']:
        deployment_uid = deployment['metadata']['guid']
        break

if deployment_uid is None:
    print("Deploying model...")

    deployment = wml_client.deployments.create(artifact_uid=model_uid, name=DEPLOYMENT_NAME, asynchronous=False)
    deployment_uid = wml_client.deployments.get_uid(deployment)
    
print("Model id: {}".format(model_uid))
print("Deployment id: {}".format(deployment_uid))

Deploying model...


#######################################################################################

Synchronous deployment creation for uid: 'fe7a19f2-7550-4a2f-b02e-6cb5c11a3cf0' started

#######################################################################################


INITIALIZING
DEPLOY_SUCCESS


------------------------------------------------------------------------------------------------
Successfully finished deployment creation, deployment_uid='cebce86d-9e6e-4f01-b789-22d8b1696b3c'
------------------------------------------------------------------------------------------------


Model id: fe7a19f2-7550-4a2f-b02e-6cb5c11a3cf0
Deployment id: cebce86d-9e6e-4f01-b789-22d8b1696b3c


# Configure OpenScale

In [27]:
from ibm_ai_openscale import APIClient
from ibm_ai_openscale.engines import *
from ibm_ai_openscale.utils import *
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
from ibm_ai_openscale.supporting_classes.enums import *

## Create schema and datamart

In [28]:
ai_client = APIClient(aios_credentials=AIOS_CREDENTIALS)
ai_client.version

'2.1.14'

#### Note: to delete an existing data_mart, uncomment the next cell and run it.

In [ ]:
#ai_client.data_mart.delete()

### Set up datamart

In [29]:
try:
    data_mart_details = ai_client.data_mart.get_details()
    if 'internal_database' in data_mart_details and data_mart_details['internal_database']:
        if KEEP_MY_INTERNAL_POSTGRES:
            print('Using existing internal datamart.')
        else:
            if DB_CREDENTIALS is None:
                print('No postgres credentials supplied. Using existing internal datamart')
            else:
                print('Switching to external datamart')
                ai_client.data_mart.delete(force=True)
                ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS)
    else:
        print('Using existing external datamart')
except:
    if DB_CREDENTIALS is None:
        print('Setting up internal datamart')
        ai_client.data_mart.setup(internal_db=True)
    else:
        print('Setting up external datamart')
        try:
            ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS)
        except:
            print('Setup failed, trying Db2 setup')
            ai_client.data_mart.setup(db_credentials=DB_CREDENTIALS, schema=DB_CREDENTIALS['username'])

Using existing internal datamart.


In [30]:
data_mart_details = ai_client.data_mart.get_details()
data_mart_details

{'database_configuration': {},
 'internal_database': True,
 'internal_database_pool': 'compose-psql',
 'service_instance_crn': 'crn:v1:bluemix:public:aiopenscale:us-south:a/47b84451ab70b94737518f7640a9ee42:b873054a-9264-48c4-bcfe-c462ac3b8cf8::',
 'status': {'state': 'active'}}

## Bind machine learning engines

In [31]:
binding_uid = ai_client.data_mart.bindings.add('WML instance', WatsonMachineLearningInstance(WML_CREDENTIALS))
if binding_uid is None:
    binding_uid = ai_client.data_mart.bindings.get_details()['service_bindings'][0]['metadata']['guid']
bindings_details = ai_client.data_mart.bindings.get_details()
ai_client.data_mart.bindings.list()

ff6b6206-589d-436a-b80a-633d799499dc,WML instance,watson_machine_learning,2019-08-07T19:30:58.275Z
4a7f62cc-f564-4144-8af4-b9f53533d2cf,sda-wml-12-11,watson_machine_learning,2019-07-25T16:14:28.598Z


In [32]:
print(binding_uid)

ff6b6206-589d-436a-b80a-633d799499dc


In [33]:
ai_client.data_mart.bindings.list_assets()

fe7a19f2-7550-4a2f-b02e-6cb5c11a3cf0,Spark German Risk Model - Final,2019-08-07T19:30:30.628Z,model,mllib-2.1,ff6b6206-589d-436a-b80a-633d799499dc,False
45c4f674-6454-4362-8af5-f8a029466c3d,Spark German Risk Model - Final,2019-07-16T21:32:03.694Z,model,mllib-2.1,4a7f62cc-f564-4144-8af4-b9f53533d2cf,False
066c8ead-6358-454e-9905-02861957b135,AIOS Spark German Risk Model - Final,2019-02-28T13:00:19.462Z,model,mllib-2.1,4a7f62cc-f564-4144-8af4-b9f53533d2cf,False
6d4e7b60-cced-42ea-abfe-cefc8b774ed5,PredictWildfireIntensity-2-25,2019-02-25T20:58:38.254Z,model,wml-1.1,4a7f62cc-f564-4144-8af4-b9f53533d2cf,False
3238b598-9e2b-48d2-b678-b3bdf19c54a3,Customer Churn Prediction Model,2018-12-11T21:53:47.763Z,model,scikit-learn-0.19,4a7f62cc-f564-4144-8af4-b9f53533d2cf,False


## Subscriptions

### Remove existing credit risk subscriptions

In [34]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
for subscription in subscriptions_uids:
    sub_name = ai_client.data_mart.subscriptions.get_details(subscription)['entity']['asset']['name']
    if sub_name == MODEL_NAME:
        ai_client.data_mart.subscriptions.delete(subscription)
        print('Deleted existing subscription for', MODEL_NAME)

Deleted existing subscription for Spark German Risk Model - Final


In [35]:
subscription = ai_client.data_mart.subscriptions.add(WatsonMachineLearningAsset(
    model_uid,
    problem_type=ProblemType.BINARY_CLASSIFICATION,
    input_data_type=InputDataType.STRUCTURED,
    label_column='Risk',
    prediction_column='predictedLabel',
    probability_column='probability',
    feature_columns = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"],
    categorical_columns = ["CheckingStatus","CreditHistory","LoanPurpose","ExistingSavings","EmploymentDuration","Sex","OthersOnLoan","OwnsProperty","InstallmentPlans","Housing","Job","Telephone","ForeignWorker"]
))

if subscription is None:
    print('Subscription already exists; get the existing one')
    subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
    for sub in subscriptions_uids:
        if ai_client.data_mart.subscriptions.get_details(sub)['entity']['asset']['name'] == MODEL_NAME:
            subscription = ai_client.data_mart.subscriptions.get(sub)

Get subscription list

In [36]:
subscriptions_uids = ai_client.data_mart.subscriptions.get_uids()
ai_client.data_mart.subscriptions.list()

c250dbc9-9d55-40dc-880f-73e5bbd4fbcf,Spark German Risk Model - Final,model,ff6b6206-589d-436a-b80a-633d799499dc,2019-08-07T19:31:18.441Z


In [37]:
subscription.get_details()

{'entity': {'asset': {'asset_id': 'fe7a19f2-7550-4a2f-b02e-6cb5c11a3cf0',
   'asset_type': 'model',
   'created_at': '2019-08-07T19:30:30.628Z',
   'name': 'Spark German Risk Model - Final',
   'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/ff6b6206-589d-436a-b80a-633d799499dc/published_models/fe7a19f2-7550-4a2f-b02e-6cb5c11a3cf0'},
  'asset_properties': {'categorical_fields': ['CheckingStatus',
    'CreditHistory',
    'LoanPurpose',
    'ExistingSavings',
    'EmploymentDuration',
    'Sex',
    'OthersOnLoan',
    'OwnsProperty',
    'InstallmentPlans',
    'Housing',
    'Job',
    'Telephone',
    'ForeignWorker'],
   'feature_fields': ['CheckingStatus',
    'LoanDuration',
    'CreditHistory',
    'LoanPurpose',
    'LoanAmount',
    'ExistingSavings',
    'EmploymentDuration',
    'InstallmentPercent',
    'Sex',
    'OthersOnLoan',
    'CurrentResidenceDuration',
    'OwnsProperty',
    'Age',
    'InstallmentPlans',
    'Housing',
    'ExistingCreditsCount',
    'J

### Score the model so we can configure monitors

In [38]:
credit_risk_scoring_endpoint = None
print(deployment_uid)

for deployment in wml_client.deployments.get_details()['resources']:
    if deployment_uid in deployment['metadata']['guid']:
        credit_risk_scoring_endpoint = deployment['entity']['scoring_url']
        
print(credit_risk_scoring_endpoint)

cebce86d-9e6e-4f01-b789-22d8b1696b3c
https://us-south.ml.cloud.ibm.com/v3/wml_instances/ff6b6206-589d-436a-b80a-633d799499dc/deployments/cebce86d-9e6e-4f01-b789-22d8b1696b3c/online


In [39]:
fields = ["CheckingStatus","LoanDuration","CreditHistory","LoanPurpose","LoanAmount","ExistingSavings","EmploymentDuration","InstallmentPercent","Sex","OthersOnLoan","CurrentResidenceDuration","OwnsProperty","Age","InstallmentPlans","Housing","ExistingCreditsCount","Job","Dependents","Telephone","ForeignWorker"]
values = [
  ["no_checking",13,"credits_paid_to_date","car_new",1343,"100_to_500","1_to_4",2,"female","none",3,"savings_insurance",46,"none","own",2,"skilled",1,"none","yes"],
  ["no_checking",24,"prior_payments_delayed","furniture",4567,"500_to_1000","1_to_4",4,"male","none",4,"savings_insurance",36,"none","free",2,"management_self-employed",1,"none","yes"],
  ["0_to_200",26,"all_credits_paid_back","car_new",863,"less_100","less_1",2,"female","co-applicant",2,"real_estate",38,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",14,"no_credits","car_new",2368,"less_100","1_to_4",3,"female","none",3,"real_estate",29,"none","own",1,"skilled",1,"none","yes"],
  ["0_to_200",4,"no_credits","car_new",250,"less_100","unemployed",2,"female","none",3,"real_estate",23,"none","rent",1,"management_self-employed",1,"none","yes"],
  ["no_checking",17,"credits_paid_to_date","car_new",832,"100_to_500","1_to_4",2,"male","none",2,"real_estate",42,"none","own",1,"skilled",1,"none","yes"],
  ["no_checking",33,"outstanding_credit","appliances",5696,"unknown","greater_7",4,"male","co-applicant",4,"unknown",54,"none","free",2,"skilled",1,"yes","yes"],
  ["0_to_200",13,"prior_payments_delayed","retraining",1375,"100_to_500","4_to_7",3,"male","none",3,"real_estate",37,"none","own",2,"management_self-employed",1,"none","yes"]
]

payload_scoring = {"fields": fields,"values": values}
scoring_response = wml_client.deployments.score(credit_risk_scoring_endpoint, payload_scoring)

print(scoring_response)

{'fields': ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'CheckingStatus_IX', 'CreditHistory_IX', 'EmploymentDuration_IX', 'ExistingSavings_IX', 'ForeignWorker_IX', 'Housing_IX', 'InstallmentPlans_IX', 'Job_IX', 'LoanPurpose_IX', 'OthersOnLoan_IX', 'OwnsProperty_IX', 'Sex_IX', 'Telephone_IX', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'], 'values': [['no_checking', 13, 'credits_paid_to_date', 'car_new', 1343, '100_to_500', '1_to_4', 2, 'female', 'none', 3, 'savings_insurance', 46, 'none', 'own', 2, 'skilled', 1, 'none', 'yes', 0.0, 1.0, 0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, [21, [1, 3, 5, 13, 14, 15, 16, 17, 18, 19, 20], [1.0, 1.0, 1.0, 13.0, 1343.0, 2.0, 3.0, 13.0, 

## Quality and feedback monitoring

### Enable quality monitoring

Wait twenty seconds to allow the payload logging table to be set up before we begin enabling monitors.

In [41]:
time.sleep(20)
subscription.quality_monitoring.enable(threshold=0.7, min_records=50)

### Feedback logging

In [42]:
!rm additional_feedback_data.json
!wget https://raw.githubusercontent.com/IBM/monitor-wml-model-with-watson-openscale/master/data/additional_feedback_data.json

rm: cannot remove ‘additional_feedback_data.json’: No such file or directory
--2019-08-07 19:33:02--  https://raw.githubusercontent.com/IBM/monitor-wml-model-with-watson-openscale/master/data/additional_feedback_data.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 16506 (16K) [text/plain]
Saving to: ‘additional_feedback_data.json’

100%[======================================>] 16,506      --.-K/s   in 0.002s  

2019-08-07 19:33:02 (9.83 MB/s) - ‘additional_feedback_data.json’ saved [16506/16506]



In [43]:
with open('additional_feedback_data.json') as feedback_file:
    additional_feedback_data = json.load(feedback_file)
subscription.feedback_logging.store(additional_feedback_data['data'])

In [44]:
subscription.feedback_logging.show_table()

less_0,10,all_credits_paid_back,car_new,250,500_to_1000,4_to_7,3,male,none,2,real_estate,23,none,rent,1,skilled,1,none,yes,No Risk,2019-08-07 19:33:06.625540+00:00
no_checking,23,prior_payments_delayed,appliances,6964,100_to_500,4_to_7,4,female,none,3,car_other,39,none,own,1,skilled,1,none,yes,Risk,2019-08-07 19:33:06.625540+00:00
0_to_200,30,outstanding_credit,appliances,3464,100_to_500,greater_7,3,male,guarantor,4,savings_insurance,51,stores,free,1,skilled,1,yes,yes,Risk,2019-08-07 19:33:06.625540+00:00
no_checking,23,outstanding_credit,car_used,2681,500_to_1000,greater_7,4,male,none,3,car_other,33,stores,free,1,unskilled,1,yes,yes,No Risk,2019-08-07 19:33:06.625540+00:00
0_to_200,18,prior_payments_delayed,furniture,1673,less_100,1_to_4,2,male,none,3,car_other,30,none,own,2,skilled,1,none,yes,Risk,2019-08-07 19:33:06.625540+00:00
no_checking,44,outstanding_credit,radio_tv,3476,unknown,greater_7,4,male,co-applicant,4,unknown,60,none,free,2,skilled,2,yes,yes,Risk,2019-08-07 19:33:06.625540+00:00
less_0,8,no_credits,education,803,less_100,unemployed,1,male,none,1,savings_insurance,19,stores,rent,1,skilled,1,none,yes,No Risk,2019-08-07 19:33:06.625540+00:00
0_to_200,7,all_credits_paid_back,car_new,250,less_100,unemployed,1,male,none,1,real_estate,19,stores,rent,1,skilled,1,none,yes,No Risk,2019-08-07 19:33:06.625540+00:00
0_to_200,33,credits_paid_to_date,radio_tv,3548,100_to_500,1_to_4,3,male,none,4,car_other,28,none,own,2,skilled,1,yes,yes,Risk,2019-08-07 19:33:06.625540+00:00
no_checking,24,prior_payments_delayed,retraining,4158,100_to_500,greater_7,3,female,none,2,savings_insurance,35,stores,own,1,unskilled,2,none,yes,Risk,2019-08-07 19:33:06.625540+00:00


In [45]:
run_details = subscription.quality_monitoring.run()
status = run_details['status']
id = run_details['id']
print(id)

print("Run status: {}".format(status))

start_time = time.time()
elapsed_time = 0

while status != 'completed' and elapsed_time < 60:
    time.sleep(10)
    run_details = subscription.quality_monitoring.get_run_details(run_uid=id)
    status = run_details['status']
    elapsed_time = time.time() - start_time
    print("Run status: {}".format(status))

1f7efdb0-aeca-4b22-ae69-effc54847a36
Run status: initializing
Run status: running
Run status: running
Run status: running
Run status: running
Run status: running
Run status: running


In [46]:
subscription.quality_monitoring.get_run_details()

{'evaluations': [{'stages': [{'name': 'Prerequisite Check',
     'completed_at': '2019-08-07T19:33:14.635Z',
     'started_at': '2019-08-07T19:33:14.310Z',
     'id': 1,
     'properties': {'training_columns': ['CheckingStatus',
       'LoanDuration',
       'CreditHistory',
       'LoanPurpose',
       'LoanAmount',
       'ExistingSavings',
       'EmploymentDuration',
       'InstallmentPercent',
       'Sex',
       'OthersOnLoan',
       'CurrentResidenceDuration',
       'OwnsProperty',
       'Age',
       'InstallmentPlans',
       'Housing',
       'ExistingCreditsCount',
       'Job',
       'Dependents',
       'Telephone',
       'ForeignWorker',
       'Risk'],
      'input_columns': ['CheckingStatus',
       'LoanDuration',
       'CreditHistory',
       'LoanPurpose',
       'LoanAmount',
       'ExistingSavings',
       'EmploymentDuration',
       'InstallmentPercent',
       'Sex',
       'OthersOnLoan',
       'CurrentResidenceDuration',
       'OwnsProperty',
      

In [47]:
subscription.quality_monitoring.show_table()

2019-08-07 19:33:14.310000+00:00,true_positive_rate,8dde3502-46f3-4ed0-90f2-8b3dda823c3b,0.42424242424242425,,,model_type: original,ff6b6206-589d-436a-b80a-633d799499dc,c250dbc9-9d55-40dc-880f-73e5bbd4fbcf,cebce86d-9e6e-4f01-b789-22d8b1696b3c
2019-08-07 19:33:14.310000+00:00,area_under_roc,8dde3502-46f3-4ed0-90f2-8b3dda823c3b,0.6736596736596737,0.7,,model_type: original,ff6b6206-589d-436a-b80a-633d799499dc,c250dbc9-9d55-40dc-880f-73e5bbd4fbcf,cebce86d-9e6e-4f01-b789-22d8b1696b3c
2019-08-07 19:33:14.310000+00:00,precision,8dde3502-46f3-4ed0-90f2-8b3dda823c3b,0.7368421052631579,,,model_type: original,ff6b6206-589d-436a-b80a-633d799499dc,c250dbc9-9d55-40dc-880f-73e5bbd4fbcf,cebce86d-9e6e-4f01-b789-22d8b1696b3c
2019-08-07 19:33:14.310000+00:00,f1_measure,8dde3502-46f3-4ed0-90f2-8b3dda823c3b,0.5384615384615385,,,model_type: original,ff6b6206-589d-436a-b80a-633d799499dc,c250dbc9-9d55-40dc-880f-73e5bbd4fbcf,cebce86d-9e6e-4f01-b789-22d8b1696b3c
2019-08-07 19:33:14.310000+00:00,accuracy,8dde3502-46f3-4ed0-90f2-8b3dda823c3b,0.7551020408163265,,,model_type: original,ff6b6206-589d-436a-b80a-633d799499dc,c250dbc9-9d55-40dc-880f-73e5bbd4fbcf,cebce86d-9e6e-4f01-b789-22d8b1696b3c
2019-08-07 19:33:14.310000+00:00,log_loss,8dde3502-46f3-4ed0-90f2-8b3dda823c3b,0.4394866071625654,,,model_type: original,ff6b6206-589d-436a-b80a-633d799499dc,c250dbc9-9d55-40dc-880f-73e5bbd4fbcf,cebce86d-9e6e-4f01-b789-22d8b1696b3c
2019-08-07 19:33:14.310000+00:00,false_positive_rate,8dde3502-46f3-4ed0-90f2-8b3dda823c3b,0.07692307692307693,,,model_type: original,ff6b6206-589d-436a-b80a-633d799499dc,c250dbc9-9d55-40dc-880f-73e5bbd4fbcf,cebce86d-9e6e-4f01-b789-22d8b1696b3c
2019-08-07 19:33:14.310000+00:00,area_under_pr,8dde3502-46f3-4ed0-90f2-8b3dda823c3b,0.6216596686521498,,,model_type: original,ff6b6206-589d-436a-b80a-633d799499dc,c250dbc9-9d55-40dc-880f-73e5bbd4fbcf,cebce86d-9e6e-4f01-b789-22d8b1696b3c
2019-08-07 19:33:14.310000+00:00,recall,8dde3502-46f3-4ed0-90f2-8b3dda823c3b,0.42424242424242425,,,model_type: original,ff6b6206-589d-436a-b80a-633d799499dc,c250dbc9-9d55-40dc-880f-73e5bbd4fbcf,cebce86d-9e6e-4f01-b789-22d8b1696b3c


In [48]:
ai_client.data_mart.get_deployment_metrics()

{'deployment_metrics': [{'subscription': {'subscription_id': 'c250dbc9-9d55-40dc-880f-73e5bbd4fbcf',
    'url': '/v1/data_marts/b873054a-9264-48c4-bcfe-c462ac3b8cf8/service_bindings/ff6b6206-589d-436a-b80a-633d799499dc/subscriptions/c250dbc9-9d55-40dc-880f-73e5bbd4fbcf'},
   'asset': {'name': 'Spark German Risk Model - Final',
    'asset_id': 'fe7a19f2-7550-4a2f-b02e-6cb5c11a3cf0',
    'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/ff6b6206-589d-436a-b80a-633d799499dc/published_models/fe7a19f2-7550-4a2f-b02e-6cb5c11a3cf0',
    'asset_type': 'model',
    'created_at': '2019-08-07T19:30:30.628Z'},
   'deployment': {'name': 'Spark German Risk Deployment - Final',
    'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/ff6b6206-589d-436a-b80a-633d799499dc/deployments/cebce86d-9e6e-4f01-b789-22d8b1696b3c',
    'deployment_type': 'online',
    'scoring_endpoint': {'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/ff6b6206-589d-436a-b80a-633d799499dc/deployments/ce

## Fairness monitoring

In [49]:
subscription.fairness_monitoring.enable(
            features=[
                Feature("Sex", majority=['male'], minority=['female'], threshold=0.95),
                Feature("Age", majority=[[26,75]], minority=[[18,25]], threshold=0.95)
            ],
            favourable_classes=['No Risk'],
            unfavourable_classes=['Risk'],
            min_records=1000,
            training_data=pd_data
        )

## Score the model again now that monitoring is configured

In [50]:
!rm german_credit_feed.json
!wget https://raw.githubusercontent.com/IBM/monitor-wml-model-with-watson-openscale/master/data/german_credit_feed.json

rm: cannot remove ‘german_credit_feed.json’: No such file or directory
--2019-08-07 19:35:39--  https://raw.githubusercontent.com/IBM/monitor-wml-model-with-watson-openscale/master/data/german_credit_feed.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3076547 (2.9M) [text/plain]
Saving to: ‘german_credit_feed.json’

100%[======================================>] 3,076,547   --.-K/s   in 0.07s   

2019-08-07 19:35:39 (41.3 MB/s) - ‘german_credit_feed.json’ saved [3076547/3076547]



Score 1000 randomly chosen records

In [51]:
import random

with open('german_credit_feed.json', 'r') as scoring_file:
    scoring_data = json.load(scoring_file)

fields = scoring_data['fields']
values = []
for _ in range(1000):
    values.append(random.choice(scoring_data['values']))
payload_scoring = {"fields": fields, "values": values}

scoring_response = wml_client.deployments.score(credit_risk_scoring_endpoint, payload_scoring)
print(scoring_response)

{'fields': ['CheckingStatus', 'LoanDuration', 'CreditHistory', 'LoanPurpose', 'LoanAmount', 'ExistingSavings', 'EmploymentDuration', 'InstallmentPercent', 'Sex', 'OthersOnLoan', 'CurrentResidenceDuration', 'OwnsProperty', 'Age', 'InstallmentPlans', 'Housing', 'ExistingCreditsCount', 'Job', 'Dependents', 'Telephone', 'ForeignWorker', 'CheckingStatus_IX', 'CreditHistory_IX', 'EmploymentDuration_IX', 'ExistingSavings_IX', 'ForeignWorker_IX', 'Housing_IX', 'InstallmentPlans_IX', 'Job_IX', 'LoanPurpose_IX', 'OthersOnLoan_IX', 'OwnsProperty_IX', 'Sex_IX', 'Telephone_IX', 'features', 'rawPrediction', 'probability', 'prediction', 'predictedLabel'], 'values': [['greater_200', 19, 'credits_paid_to_date', 'furniture', 3184, 'less_100', 'less_1', 3, 'male', 'none', 3, 'real_estate', 37, 'none', 'free', 1, 'management_self-employed', 1, 'none', 'yes', 3.0, 1.0, 3.0, 0.0, 0.0, 2.0, 0.0, 2.0, 1.0, 0.0, 2.0, 0.0, 0.0, [3.0, 1.0, 1.0, 0.0, 3.0, 0.0, 0.0, 2.0, 0.0, 2.0, 2.0, 0.0, 0.0, 19.0, 3184.0, 3.0,

In [52]:
subscription.get_details()

{'entity': {'asset': {'asset_id': 'fe7a19f2-7550-4a2f-b02e-6cb5c11a3cf0',
   'asset_type': 'model',
   'created_at': '2019-08-07T19:30:30.628Z',
   'name': 'Spark German Risk Model - Final',
   'url': 'https://us-south.ml.cloud.ibm.com/v3/wml_instances/ff6b6206-589d-436a-b80a-633d799499dc/published_models/fe7a19f2-7550-4a2f-b02e-6cb5c11a3cf0'},
  'asset_properties': {'categorical_fields': ['CheckingStatus',
    'CreditHistory',
    'LoanPurpose',
    'ExistingSavings',
    'EmploymentDuration',
    'Sex',
    'OthersOnLoan',
    'OwnsProperty',
    'InstallmentPlans',
    'Housing',
    'Job',
    'Telephone',
    'ForeignWorker'],
   'feature_fields': ['CheckingStatus',
    'LoanDuration',
    'CreditHistory',
    'LoanPurpose',
    'LoanAmount',
    'ExistingSavings',
    'EmploymentDuration',
    'InstallmentPercent',
    'Sex',
    'OthersOnLoan',
    'CurrentResidenceDuration',
    'OwnsProperty',
    'Age',
    'InstallmentPlans',
    'Housing',
    'ExistingCreditsCount',
    'J

# Insert historical payloads

In [53]:
!rm payload_history*.json
!wget https://raw.githubusercontent.com/IBM/monitor-wml-model-with-watson-openscale/master/data/payload_history_1.json
!wget https://raw.githubusercontent.com/IBM/monitor-wml-model-with-watson-openscale/master/data/payload_history_2.json
!wget https://raw.githubusercontent.com/IBM/monitor-wml-model-with-watson-openscale/master/data/payload_history_3.json
!wget https://raw.githubusercontent.com/IBM/monitor-wml-model-with-watson-openscale/master/data/payload_history_4.json
!wget https://raw.githubusercontent.com/IBM/monitor-wml-model-with-watson-openscale/master/data/payload_history_5.json
!wget https://raw.githubusercontent.com/IBM/monitor-wml-model-with-watson-openscale/master/data/payload_history_6.json
!wget https://raw.githubusercontent.com/IBM/monitor-wml-model-with-watson-openscale/master/data/payload_history_7.json

rm: cannot remove ‘payload_history*.json’: No such file or directory
--2019-08-07 19:36:03--  https://raw.githubusercontent.com/IBM/monitor-wml-model-with-watson-openscale/master/data/payload_history_1.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3817949 (3.6M) [text/plain]
Saving to: ‘payload_history_1.json’

100%[======================================>] 3,817,949   --.-K/s   in 0.08s   

2019-08-07 19:36:03 (44.0 MB/s) - ‘payload_history_1.json’ saved [3817949/3817949]

--2019-08-07 19:36:03--  https://raw.githubusercontent.com/IBM/monitor-wml-model-with-watson-openscale/master/data/payload_history_2.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HT

In [54]:
historyDays = 7

In [55]:
from ibm_ai_openscale.supporting_classes import PayloadRecord, Feature
import datetime
import time

for day in range(historyDays):
    print('Loading day {}'.format(day + 1))
    history_file = 'payload_history_' + str(day + 1) + '.json'
    with open(history_file) as f:
        payloads = json.load(f)
        hourly_records = int(len(payloads) / 24)
        index = 0
        for hour in range(24):
            recordsList = []
            for i in range(hourly_records):
                score_time = str(datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1))))
                recordsList.append(PayloadRecord(request=payloads[index]['request'], response=payloads[index]['response'], scoring_timestamp=score_time))
                index += 1
            subscription.payload_logging.store(records=recordsList)
print('Finished')

Loading day 1
Loading day 2
Loading day 3
Loading day 4
Loading day 5
Loading day 6
Loading day 7
Finished


In [56]:
data_mart_id = subscription.get_details()['metadata']['url'].split('/service_bindings')[0].split('marts/')[1]
print(data_mart_id)

b873054a-9264-48c4-bcfe-c462ac3b8cf8


In [57]:
performance_metrics_url = 'https://api.aiopenscale.cloud.ibm.com' + subscription.get_details()['metadata']['url'].split('/service_bindings')[0] + '/metrics'
print(performance_metrics_url)

https://api.aiopenscale.cloud.ibm.com/v1/data_marts/b873054a-9264-48c4-bcfe-c462ac3b8cf8/metrics


## Insert historical fairness metrics

In [58]:
!rm fairness_history.json
!wget https://raw.githubusercontent.com/IBM/monitor-wml-model-with-watson-openscale/master/data/fairness_history.json

rm: cannot remove ‘fairness_history.json’: No such file or directory
--2019-08-07 19:37:10--  https://raw.githubusercontent.com/IBM/monitor-wml-model-with-watson-openscale/master/data/fairness_history.json
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 199.232.8.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|199.232.8.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 877247 (857K) [text/plain]
Saving to: ‘fairness_history.json’

100%[======================================>] 877,247     --.-K/s   in 0.04s   

2019-08-07 19:37:11 (23.0 MB/s) - ‘fairness_history.json’ saved [877247/877247]



In [59]:
import random
token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': AIOS_CREDENTIALS['apikey']
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

with open('fairness_history.json', 'r') as history_file:
    payloads = json.load(history_file)

for day in range(historyDays):
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        
        qualityMetric = {
            'metric_type': 'fairness',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': random.choice(payloads)
        }

        response = requests.post(performance_metrics_url, json=[qualityMetric], headers=iam_headers)
print('Finished')

Day 1
Day 2
Day 3
Day 4
Day 5
Day 6
Day 7
Finished


## Insert historical quality metrics

In [60]:
token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': AIOS_CREDENTIALS['apikey']
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

measurements = [0.76, 0.78, 0.68, 0.72, 0.73, 0.77, 0.80]
for day in range(historyDays):
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        
        qualityMetric = {
            'metric_type': 'quality',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': {
                'quality': measurements[day],
                'threshold': 0.7,
                'metrics': [
                    {
                        'name': 'auroc',
                        'value': measurements[day],
                        'threshold': 0.7
                    }
                ]
            }
        }

        response = requests.post(performance_metrics_url, json=[qualityMetric], headers=iam_headers)
print('Finished')

Day 1
Day 2
Day 3
Day 4
Day 5
Day 6
Day 7
Finished


## Insert historical performance metrics

In [61]:
token_data = {
    'grant_type': 'urn:ibm:params:oauth:grant-type:apikey',
    'response_type': 'cloud_iam',
    'apikey': AIOS_CREDENTIALS['apikey']
}

response = requests.post('https://iam.bluemix.net/identity/token', data=token_data)
iam_token = response.json()['access_token']
iam_headers = {
    'Content-Type': 'application/json',
    'Authorization': 'Bearer %s' % iam_token
}

for day in range(historyDays):
    print('Day', day + 1)
    for hour in range(24):
        score_time = (datetime.datetime.utcnow() + datetime.timedelta(hours=(-(24*day + hour + 1)))).strftime('%Y-%m-%dT%H:%M:%SZ')
        score_count = random.randint(60, 600)
        score_resp = random.uniform(60, 300)

        performanceMetric = {
            'metric_type': 'performance',
            'binding_id': binding_uid,
            'timestamp': score_time,
            'subscription_id': model_uid,
            'asset_revision': model_uid,
            'deployment_id': deployment_uid,
            'value': {
                'response_time': score_resp,
                'records': score_count
            }
        }

        response = requests.post(performance_metrics_url, json=[performanceMetric], headers=iam_headers)
print('Finished')

Day 1
Day 2
Day 3
Day 4
Day 5
Day 6
Day 7
Finished


## Configure Explainability

In [62]:
from ibm_ai_openscale.supporting_classes import *
subscription.explainability.enable(training_data=pd_data)

In [63]:
subscription.explainability.get_details()

{'enabled': True,
 'parameters': {'training_statistics': {'base_values': {'0': 'no_checking',
    '1': 21.0,
    '10': 3.0,
    '11': 'savings_insurance',
    '12': 36.0,
    '13': 'none',
    '14': 'own',
    '15': 1.0,
    '16': 'skilled',
    '17': 1.0,
    '18': 'none',
    '19': 'yes',
    '2': 'prior_payments_delayed',
    '3': 'car_new',
    '4': 3238.5,
    '5': 'less_100',
    '6': '1_to_4',
    '7': 3.0,
    '8': 'male',
    '9': 'none'},
   'categorical_columns': ['CheckingStatus',
    'CreditHistory',
    'LoanPurpose',
    'ExistingSavings',
    'EmploymentDuration',
    'Sex',
    'OthersOnLoan',
    'OwnsProperty',
    'InstallmentPlans',
    'Housing',
    'Job',
    'Telephone',
    'ForeignWorker'],
   'categorical_columns_encoding_mapping': {'0': ['0_to_200',
     'greater_200',
     'less_0',
     'no_checking'],
    '11': ['car_other', 'real_estate', 'savings_insurance', 'unknown'],
    '13': ['bank', 'none', 'stores'],
    '14': ['free', 'own', 'rent'],
    '16': 

## Run fairness monitor

Kick off a fairness monitor run on current data. Depending on how fast the monitor runs, the table may not contain the most recent results.

In [64]:
run_details = subscription.fairness_monitoring.run()

In [65]:
subscription.fairness_monitoring.show_table()

2019-08-07 17:37:13+00:00,Sex,female,True,0.938,76.0,ff6b6206-589d-436a-b80a-633d799499dc,c250dbc9-9d55-40dc-880f-73e5bbd4fbcf,fe7a19f2-7550-4a2f-b02e-6cb5c11a3cf0,cebce86d-9e6e-4f01-b789-22d8b1696b3c,
2019-08-07 17:37:13+00:00,Age,"[18, 25]",False,1.009,84.31372549019608,ff6b6206-589d-436a-b80a-633d799499dc,c250dbc9-9d55-40dc-880f-73e5bbd4fbcf,fe7a19f2-7550-4a2f-b02e-6cb5c11a3cf0,cebce86d-9e6e-4f01-b789-22d8b1696b3c,
2019-08-07 16:37:13+00:00,Sex,female,True,0.925,74.0,ff6b6206-589d-436a-b80a-633d799499dc,c250dbc9-9d55-40dc-880f-73e5bbd4fbcf,fe7a19f2-7550-4a2f-b02e-6cb5c11a3cf0,cebce86d-9e6e-4f01-b789-22d8b1696b3c,
2019-08-07 16:37:13+00:00,Age,"[18, 25]",False,1.022,84.40111420612814,ff6b6206-589d-436a-b80a-633d799499dc,c250dbc9-9d55-40dc-880f-73e5bbd4fbcf,fe7a19f2-7550-4a2f-b02e-6cb5c11a3cf0,cebce86d-9e6e-4f01-b789-22d8b1696b3c,
2019-08-07 18:37:13+00:00,Sex,female,False,0.96,72.5,ff6b6206-589d-436a-b80a-633d799499dc,c250dbc9-9d55-40dc-880f-73e5bbd4fbcf,fe7a19f2-7550-4a2f-b02e-6cb5c11a3cf0,cebce86d-9e6e-4f01-b789-22d8b1696b3c,
2019-08-07 18:37:13+00:00,Age,"[18, 25]",False,0.965,77.33711048158641,ff6b6206-589d-436a-b80a-633d799499dc,c250dbc9-9d55-40dc-880f-73e5bbd4fbcf,fe7a19f2-7550-4a2f-b02e-6cb5c11a3cf0,cebce86d-9e6e-4f01-b789-22d8b1696b3c,
2019-08-07 15:37:13+00:00,Sex,female,False,0.962,75.5,ff6b6206-589d-436a-b80a-633d799499dc,c250dbc9-9d55-40dc-880f-73e5bbd4fbcf,fe7a19f2-7550-4a2f-b02e-6cb5c11a3cf0,cebce86d-9e6e-4f01-b789-22d8b1696b3c,
2019-08-07 15:37:13+00:00,Age,"[18, 25]",False,1.018,83.75350140056022,ff6b6206-589d-436a-b80a-633d799499dc,c250dbc9-9d55-40dc-880f-73e5bbd4fbcf,fe7a19f2-7550-4a2f-b02e-6cb5c11a3cf0,cebce86d-9e6e-4f01-b789-22d8b1696b3c,
2019-08-07 14:37:13+00:00,Sex,female,True,0.925,67.5,ff6b6206-589d-436a-b80a-633d799499dc,c250dbc9-9d55-40dc-880f-73e5bbd4fbcf,fe7a19f2-7550-4a2f-b02e-6cb5c11a3cf0,cebce86d-9e6e-4f01-b789-22d8b1696b3c,
2019-08-07 14:37:13+00:00,Age,"[18, 25]",False,1.039,79.17808219178082,ff6b6206-589d-436a-b80a-633d799499dc,c250dbc9-9d55-40dc-880f-73e5bbd4fbcf,fe7a19f2-7550-4a2f-b02e-6cb5c11a3cf0,cebce86d-9e6e-4f01-b789-22d8b1696b3c,


## Additional data to help debugging
You will use the `Deployment` to configure the OpenScale dashboard.

In [66]:
print('Datamart:', data_mart_id)
print('Model:', model_uid)
print('Deployment:', deployment_uid)
print('Binding:', binding_uid)
print('Scoring URL:', credit_risk_scoring_endpoint)

Datamart: b873054a-9264-48c4-bcfe-c462ac3b8cf8
Model: fe7a19f2-7550-4a2f-b02e-6cb5c11a3cf0
Deployment: cebce86d-9e6e-4f01-b789-22d8b1696b3c
Binding: ff6b6206-589d-436a-b80a-633d799499dc
Scoring URL: https://us-south.ml.cloud.ibm.com/v3/wml_instances/ff6b6206-589d-436a-b80a-633d799499dc/deployments/cebce86d-9e6e-4f01-b789-22d8b1696b3c/online


## Identify transactions for Explainability

Transaction IDs identified by the cells below can be copied and pasted into the Explainability tab of the OpenScale dashboard.

In [67]:
payload_data = subscription.payload_logging.get_table_content(limit=60)
payload_data.filter(items=['scoring_id', 'predictedLabel', 'probability'])

,scoring_id,predictedLabel,probability
0,1c21b2ed64b323b1de386fcf3ff4dbff-1,No Risk,"[0.8528619400512564, 0.14713805994874352]"
1,1c21b2ed64b323b1de386fcf3ff4dbff-10,Risk,"[0.4634338952303577, 0.5365661047696423]"
2,1c21b2ed64b323b1de386fcf3ff4dbff-100,Risk,"[0.4380544117391884, 0.5619455882608115]"
3,1c21b2ed64b323b1de386fcf3ff4dbff-1000,No Risk,"[0.9678965149050545, 0.032103485094945636]"
4,1c21b2ed64b323b1de386fcf3ff4dbff-101,Risk,"[0.3758789886449652, 0.6241210113550348]"
5,1c21b2ed64b323b1de386fcf3ff4dbff-102,No Risk,"[0.510120022100116, 0.489879977899884]"
6,1c21b2ed64b323b1de386fcf3ff4dbff-103,No Risk,"[0.5906681557039747, 0.40933184429602526]"
7,1c21b2ed64b323b1de386fcf3ff4dbff-104,No Risk,"[0.8356934588822307, 0.16430654111776932]"
8,1c21b2ed64b323b1de386fcf3ff4dbff-105,No Risk,"[0.5615068950890428, 0.4384931049109572]"
9,1c21b2ed64b323b1de386fcf3ff4dbff-106,No Risk,"[0.6824558618950868, 0.3175441381049132]"


## Congratulations!

You have finished the hands-on lab for IBM Watson OpenScale. You can now view the [OpenScale Dashboard](https://aiopenscale.cloud.ibm.com/). Click on the tile for the German Credit model to see fairness, accuracy, and performance monitors. Click on the timeseries graph to get detailed information on transactions during a specific time window.

## Next steps

OpenScale shows model performance over time. You have two options to keep data flowing to your OpenScale graphs:
  * Download, configure and schedule the [model feed notebook](https://raw.githubusercontent.com/emartensibm/german-credit/master/german_credit_scoring_feed.ipynb). This notebook can be set up with your WML credentials, and scheduled to provide a consistent flow of scoring requests to your model, which will appear in your OpenScale monitors.
  * Re-run this notebook. Running this notebook from the beginning will delete and re-create the model and deployment, and re-create the historical data. Please note that the payload and measurement logs for the previous deployment will continue to be stored in your datamart, and cal be deleted if necessary.